In [1]:
import os
import itertools
import shutil
import math
from configparser import ConfigParser
from datetime import datetime
from time import time, gmtime, strftime

import numpy as np
import pandas as pd
import tensorflow as tf
from utils import EarlyStoppingHook
#from google.cloud import storage


In [2]:
# configファイルの読み込み
config_filename = '../src/config/NeuralFM_config.ini'

param_config = ConfigParser()
param_config.read(config_filename)

key_list = ['path','train','eval','predict','model']

for parent_key in key_list:
    print(parent_key)
    for key in param_config[parent_key]:
        print(key, ':', param_config[parent_key][key])
    print('\n')


path
username : w_michishita
work_dir : /home/w_michishita/recommend_MovieLens
model_dir : /home/w_michishita/recommend_MovieLens/models/


train
username : w_michishita
work_dir : /home/w_michishita/recommend_MovieLens
filename_pattern : /home/w_michishita/recommend_MovieLens/data/preprocessed/20190501_train1.csv
batch_size : 128
max_steps : 1000000
num_epochs : 200
skip_header_lines : 1


eval
username : w_michishita
work_dir : /home/w_michishita/recommend_MovieLens
filename_pattern : /home/w_michishita/recommend_MovieLens/data/preprocessed/20190501_eval1.csv
batch_size : 128
max_steps : None
num_epochs : None
skip_header_lines : 1


predict
username : w_michishita
work_dir : /home/w_michishita/recommend_MovieLens
filename_pattern : /home/w_michishita/recommend_MovieLens/data/preprocessed/20190501_test1.csv
batch_size : 128
max_steps : None
num_epochs : None
skip_header_lines : 1


model
user_bucket_size : 2000
item_bucket_size : 3000
user_embedding_dim : 256
item_embedding_dim : 256

In [3]:
# データの入力
HEADER = ['user_id', 'item_id', 'rating']
HEADER_DEFAULTS = [['0'], ['0'], [0]]

FEATURE_NAMES = ['user_id', 'item_id']
CATEGORICAL_FEATURE_NAMES_WITH_BUCKET_SIZE = {
  'user_id': int(param_config['model']['user_bucket_size']),
  'item_id' : int(param_config['model']['item_bucket_size'])
  }

USED_FEATURE_NAMES = ['user_id', 'item_id', 'rating']

CATEGORICAL_FEATURE_NAMES =  list(CATEGORICAL_FEATURE_NAMES_WITH_BUCKET_SIZE.keys())
TARGET = 'rating'
TARGET_LABELS = [0,1]


In [4]:
user_embedding_dim = int(param_config['model']['user_embedding_dim'])
item_embedding_dim = int(param_config['model']['item_embedding_dim'])
    
# カラム情報取得
categorical_hash_user_raw = \
tf.feature_column.categorical_column_with_hash_bucket('user_id', CATEGORICAL_FEATURE_NAMES_WITH_BUCKET_SIZE['user_id'])
categorical_hash_item_raw = \
    tf.feature_column.categorical_column_with_hash_bucket('item_id', CATEGORICAL_FEATURE_NAMES_WITH_BUCKET_SIZE['item_id'])

categorical_hash_user = tf.feature_column.indicator_column(categorical_hash_user_raw)    
categorical_hash_item = tf.feature_column.indicator_column(categorical_hash_item_raw)
categorical_feature_linear = [categorical_hash_user, categorical_hash_item]
    
# 後半のEmbeddingパートを作成
categorical_feature_user_emb = tf.feature_column.embedding_column(
    categorical_column=categorical_hash_user_raw, dimension=user_embedding_dim)
categorical_feature_item_emb = tf.feature_column.embedding_column(
    categorical_column=categorical_hash_item_raw, dimension=item_embedding_dim)
categorical_feature_emb = [categorical_feature_user_emb, categorical_feature_item_emb]


In [5]:
params = {
    'categorical_feature_linear': categorical_feature_linear,
    'categorical_feature_emb': categorical_feature_emb,
    'hidden_units': [128, 64, 32],
    'dropout_prob': 0.3,
    'n_classes': 2
}

In [6]:
def NeuralFM_fn(features, labels, mode, params):
    input_features_linear = tf.feature_column.input_layer(features, params['categorical_feature_linear'])
    input_features_dnn = tf.feature_column.input_layer(features, params['categorical_feature_emb'])
    
    feature_sq_sum = tf.square(tf.reduce_sum(input_features_dnn, axis=1, keepdims=True))
    feature_sum_sq = tf.reduce_sum(tf.square(input_features_dnn), axis=1, keepdims=True)
    cross_term = 0.5 * tf.subtract(feature_sq_sum, feature_sum_sq)
    
    for hidden_unit in params['hidden_units']:
        layer = tf.layers.dense(cross_term, units=hidden_unit, activation=tf.nn.relu)
    layer = tf.nn.dropout(layer, rate=params['dropout_prob'])
    logits = tf.layers.dense(layer, params['n_classes'], activation=None)
    
    predicted_classes = tf.argmax(logits, 1)
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            'class_ids': predicted_classes[:, tf.newaxis],
            'probabilities': tf.nn.softmax(logits),
            'logits': logits,
        }
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)
    
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    
    accuracy = tf.metrics.accuracy(labels=labels,
                               predictions=predicted_classes,
                               name='acc_op')
    
    metrics = {'accuracy': accuracy}
    tf.summary.scalar('accuracy', accuracy[1])

    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(
            mode, loss=loss, eval_metric_ops=metrics)
    
    optimizer = tf.train.AdamOptimizer()
    train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)


In [7]:
def parse_input_config(config, phase):
    '''
    configをパースする関数（内部使用）
    configはいつもの設定ファイル
    phaseは{train, eval, predict}のいずれか
    batch_sizeやnum_epochsなどint型で入って欲しい変数がstrになってしまいintへの変換が必要となったためこの関数を用意した
    '''
    filename_pattern = config[phase]['filename_pattern']
    batch_size = int(config[phase]['batch_size'])
    try:
        num_epochs = int(config[phase]['num_epochs'])
    except:
        num_epochs = 1 # Noneにすると一生止まらない
    # これだけは正直共通にしておきたい
    skip_header_lines = int(config[phase]['skip_header_lines'])

    return filename_pattern, batch_size, num_epochs, skip_header_lines



In [8]:
def parse_csv_row(csv_row):
    '''
    csvをparseする関数
    途中でHEADERやHEADER_DEFAULTSを使用しているのでそれらを定義する関数か何かが必要
    （lambdaで使用しているため引数に加えることはできない）
    '''
    columns = tf.decode_csv(csv_row, record_defaults=HEADER_DEFAULTS)
    features = dict(zip(HEADER, columns))

    target = features.pop(TARGET)
    return features, target


In [9]:
def csv_input_fn(config, phase, mode=tf.estimator.ModeKeys.EVAL):
    '''
    ファイル名のパターンとか学習・評価時の設定は全てconfigに入れておいた方が管理しやすい
    （いつ・どういうタイミングで？）
    modeは直打ちで渡す方が楽そう（どこで？）
    '''
    shuffle = True if mode == tf.estimator.ModeKeys.TRAIN else False
    filename_pattern, batch_size, num_epochs, skip_header_lines = parse_input_config(config, phase)

    # ファイル名のパターンを元にデータの読み込み
    file_names = tf.matching_files(filename_pattern)
    dataset = tf.data.TextLineDataset(filenames=file_names)
    dataset = dataset.skip(skip_header_lines)

    # バッチサイズ分だけ切り出しgenerateする
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(lambda csv_row: parse_csv_row(csv_row))
    dataset = dataset.repeat(num_epochs)
    iterator = dataset.make_one_shot_iterator()

    features, target = iterator.get_next()

    return features, target

In [10]:
def json_serving_input_fn():
    '''
    serving用のinput_fn
    同じくTARGETやUSED_FEATURE_NAMEを呼び出す関数がここにも必要
    '''
    receiver_tensor = {}
    for feature_name in USED_FEATURE_NAMES:
        dtype = tf.float32 if feature_name == TARGET else tf.string
        receiver_tensor[feature_name] = tf.placeholder(shape=[None], dtype=dtype)

    return tf.estimator.export.ServingInputReceiver(receiver_tensor, receiver_tensor)


In [11]:
train_input_fn = lambda: csv_input_fn(config=param_config, 
                                      phase='train', 
                                      mode=tf.estimator.ModeKeys.TRAIN)

eval_input_fn = lambda: csv_input_fn(config=param_config,
                                     phase='eval', 
                                     mode=tf.estimator.ModeKeys.EVAL)


In [12]:
train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn,
                  max_steps=int(param_config['train']['max_steps']),
                  hooks=[EarlyStoppingHook(int(param_config['model']['early_stop']))]
                  )

eval_spec = tf.estimator.EvalSpec(input_fn=eval_input_fn,
                exporters=[tf.estimator.LatestExporter(name="estimate",  
                                                       serving_input_receiver_fn=json_serving_input_fn)],
                steps=None,
                throttle_secs = 15
                )


*** Early Stopping Hook: - Created
*** Early Stopping Hook:: Early Stopping Rounds: 2000



In [13]:
raw_execute_time = gmtime()
execute_time = strftime("%Y%m%d_%H%M%S", raw_execute_time )
model_dir = os.path.join(param_config['path']['model_dir'], execute_time)
print(model_dir)

/home/w_michishita/recommend_MovieLens/models/20190504_075559


In [14]:
run_config = tf.estimator.RunConfig().replace(model_dir=model_dir,save_checkpoints_secs=20)


In [15]:
estimator = tf.estimator.Estimator(model_fn=NeuralFM_fn, 
                                  params=params, 
                                  config=run_config)

INFO:tensorflow:Using config: {'_log_step_count_steps': 100, '_task_type': 'worker', '_global_id_in_cluster': 0, '_evaluation_master': '', '_master': '', '_keep_checkpoint_every_n_hours': 10000, '_num_worker_replicas': 1, '_experimental_distribute': None, '_train_distribute': None, '_model_dir': '/home/w_michishita/recommend_MovieLens/models/20190504_075559', '_save_checkpoints_steps': None, '_save_summary_steps': 100, '_service': None, '_save_checkpoints_secs': 20, '_is_chief': True, '_task_id': 0, '_keep_checkpoint_max': 5, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_num_ps_replicas': 0, '_eval_distribute': None, '_tf_random_seed': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4702452898>, '_protocol': None, '_device_fn': None}


In [16]:
tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 20.
Instructions for updating:
Colocations handled automatically by placer.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs 

Early Stopping Hook: No improvment! Counter: 17
************************

Early Stopping Hook: No improvment! Counter: 18
************************

Early Stopping Hook: No improvment! Counter: 19
************************

Early Stopping Hook: No improvment! Counter: 20
************************

Early Stopping Hook: No improvment! Counter: 21
************************

Early Stopping Hook: No improvment! Counter: 22
************************

Early Stopping Hook: No improvment! Counter: 23
************************

Early Stopping Hook: No improvment! Counter: 24
************************

Early Stopping Hook: No improvment! Counter: 25
************************

Early Stopping Hook: No improvment! Counter: 26
************************

Early Stopping Hook: No improvment! Counter: 27
************************

Early Stopping Hook: No improvment! Counter: 28
************************

Early Stopping Hook: No improvment! Counter: 29
************************

Early Stopping Hook: No improvment! Co

Early Stopping Hook: No improvment! Counter: 139
************************

Early Stopping Hook: No improvment! Counter: 140
************************

Early Stopping Hook: No improvment! Counter: 141
************************

Early Stopping Hook: No improvment! Counter: 142
************************

Early Stopping Hook: No improvment! Counter: 143
************************

Early Stopping Hook: No improvment! Counter: 144
************************

Early Stopping Hook: No improvment! Counter: 145
************************

Early Stopping Hook: No improvment! Counter: 146
************************

************************

Early Stopping Hook: No improvment! Counter: 1
************************

Early Stopping Hook: No improvment! Counter: 2
************************

Early Stopping Hook: No improvment! Counter: 3
************************

************************

Early Stopping Hook: No improvment! Counter: 1
************************

************************

Early Stopping Hook: No improv

Early Stopping Hook: No improvment! Counter: 6
************************

INFO:tensorflow:loss = 0.686629, step = 301 (0.781 sec)
Early Stopping Hook: No improvment! Counter: 7
************************

Early Stopping Hook: No improvment! Counter: 8
************************

Early Stopping Hook: No improvment! Counter: 9
************************

Early Stopping Hook: No improvment! Counter: 10
************************

Early Stopping Hook: No improvment! Counter: 11
************************

Early Stopping Hook: No improvment! Counter: 12
************************

Early Stopping Hook: No improvment! Counter: 13
************************

Early Stopping Hook: No improvment! Counter: 14
************************

Early Stopping Hook: No improvment! Counter: 15
************************

Early Stopping Hook: No improvment! Counter: 16
************************

Early Stopping Hook: No improvment! Counter: 17
************************

Early Stopping Hook: No improvment! Counter: 18
************

Early Stopping Hook: No improvment! Counter: 32
************************

Early Stopping Hook: No improvment! Counter: 33
************************

Early Stopping Hook: No improvment! Counter: 34
************************

Early Stopping Hook: No improvment! Counter: 35
************************

Early Stopping Hook: No improvment! Counter: 36
************************

Early Stopping Hook: No improvment! Counter: 37
************************

Early Stopping Hook: No improvment! Counter: 38
************************

Early Stopping Hook: No improvment! Counter: 39
************************

Early Stopping Hook: No improvment! Counter: 40
************************

Early Stopping Hook: No improvment! Counter: 41
************************

Early Stopping Hook: No improvment! Counter: 42
************************

Early Stopping Hook: No improvment! Counter: 43
************************

Early Stopping Hook: No improvment! Counter: 44
************************

Early Stopping Hook: No improvment! Co

...

Early Stopping Hook: No improvment! Counter: 1668
************************

Early Stopping Hook: No improvment! Counter: 1669
************************

Early Stopping Hook: No improvment! Counter: 1670
************************

Early Stopping Hook: No improvment! Counter: 1671
************************

Early Stopping Hook: No improvment! Counter: 1672
************************

Early Stopping Hook: No improvment! Counter: 1673
************************

Early Stopping Hook: No improvment! Counter: 1674
************************

Early Stopping Hook: No improvment! Counter: 1675
************************

Early Stopping Hook: No improvment! Counter: 1676
************************

Early Stopping Hook: No improvment! Counter: 1677
************************

Early Stopping Hook: No improvment! Counter: 1678
************************

Early Stopping Hook: No improvment! Counter: 1679
************************

Early Stopping Hook: No improvment! Counter: 1680
************************

Early Stoppi

Early Stopping Hook: No improvment! Counter: 1775
************************

Early Stopping Hook: No improvment! Counter: 1776
************************

Early Stopping Hook: No improvment! Counter: 1777
************************

Early Stopping Hook: No improvment! Counter: 1778
************************

Early Stopping Hook: No improvment! Counter: 1779
************************

Early Stopping Hook: No improvment! Counter: 1780
************************

Early Stopping Hook: No improvment! Counter: 1781
************************

Early Stopping Hook: No improvment! Counter: 1782
************************

Early Stopping Hook: No improvment! Counter: 1783
************************

Early Stopping Hook: No improvment! Counter: 1784
************************

Early Stopping Hook: No improvment! Counter: 1785
************************

Early Stopping Hook: No improvment! Counter: 1786
************************

Early Stopping Hook: No improvment! Counter: 1787
************************

Early Stoppi

Early Stopping Hook: No improvment! Counter: 1884
************************

Early Stopping Hook: No improvment! Counter: 1885
************************

Early Stopping Hook: No improvment! Counter: 1886
************************

Early Stopping Hook: No improvment! Counter: 1887
************************

INFO:tensorflow:Saving checkpoints for 9100 into /home/w_michishita/recommend_MovieLens/models/20190504_075559/model.ckpt.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-04T07:57:21Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/w_michishita/recommend_MovieLens/models/20190504_075559/model.ckpt-9100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-05-04-07:57:22
INFO:tensorflow:Saving dict for global step 9100: accuracy = 0.74285, global_step = 9100, loss = 0.5400887
INFO:tensorflow:Saving 'checkpoint

Early Stopping Hook: No improvment! Counter: 1970
************************

Early Stopping Hook: No improvment! Counter: 1971
************************

Early Stopping Hook: No improvment! Counter: 1972
************************

Early Stopping Hook: No improvment! Counter: 1973
************************

Early Stopping Hook: No improvment! Counter: 1974
************************

Early Stopping Hook: No improvment! Counter: 1975
************************

Early Stopping Hook: No improvment! Counter: 1976
************************

Early Stopping Hook: No improvment! Counter: 1977
************************

Early Stopping Hook: No improvment! Counter: 1978
************************

Early Stopping Hook: No improvment! Counter: 1979
************************

Early Stopping Hook: No improvment! Counter: 1980
************************

Early Stopping Hook: No improvment! Counter: 1981
************************

Early Stopping Hook: No improvment! Counter: 1982
************************

Early Stoppi

({'accuracy': 0.744125, 'global_step': 9212, 'loss': 0.5370289},
 [b'/home/w_michishita/recommend_MovieLens/models/20190504_075559/export/estimate/1556956645'])

## 予測&評価

In [17]:
test_data = pd.read_csv(param_config['predict']['filename_pattern'])
test_size = len(test_data)


In [18]:
predict_input_fn = lambda: csv_input_fn(config=param_config, 
                                      phase='predict', 
                                      mode=tf.estimator.ModeKeys.PREDICT)


In [19]:
predictions = estimator.predict(input_fn=predict_input_fn)

In [20]:
values = list(map(lambda item: item["class_ids"][0],list(itertools.islice(predictions, test_size))))


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/w_michishita/recommend_MovieLens/models/20190504_075559/model.ckpt-9212
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [21]:
from sklearn.metrics import accuracy_score, roc_auc_score

test_value = np.array(test_data.iloc[:,2])
pred_value = np.array(values)
pred_value_binary = np.round(pred_value)

auc_score = roc_auc_score(test_value, pred_value)
accuracy_score = accuracy_score(test_value, pred_value_binary)


In [29]:
print(auc_score)

0.744061859193438


In [28]:
print(accuracy_score)

0.7441


In [24]:
pred_value_binary[:20]

array([0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0])

In [25]:
test_value[:20]

array([0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1])

In [26]:
result = ','.join([config_filename,str(auc_score), str(accuracy_score),execute_time]) + '\n'

with open('../log/result.csv', 'a') as f:
    f.write(result)


In [27]:
result_df = pd.read_csv('../log/result.csv')
result_df.tail()


,filename,AUC,accuracy,param
3,../src/config/task_config_20190501_2.ini,0.729610,0.67065,20190501_060357
4,../src/config/task_config_20190501_2.ini,0.732487,0.67295,20190501_062544
5,../src/config/task_config_20190501_1.ini,0.500000,0.49840,20190503_104619
6,../src/config/NeuralFM_config.ini,0.760653,0.76045,20190504_074715
7,../src/config/NeuralFM_config.ini,0.744062,0.74410,20190504_075559
